In [1]:
!python -V

Python 3.9.7


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [14]:
url = 'https://nyc-tlc.s3.amazonaws.com/trip+data/'
files = ['green_tripdata_2021-01.parquet', 'green_tripdata_2021-02.parquet']
data_sources = [url + file for file in files]
data_destinations = ['./data/' + file for file in files]

In [25]:
for i in range(len(data_sources)):
    source = data_sources[i]
    destination = data_destinations[i]
    print(f'Downloading {data_sources[i]}')
    !curl $source -o $destination --create-dirs 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 10 1302k   10  139k    0     0   178k      0  0:00:07 --:--:--  0:00:07  179k
100 1302k  100 1302k    0     0   746k      0  0:00:01  0:00:01 --:--:--  746k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 34 1118k   34  381k    0     0   381k      0  0:00:02  0:00:01  0:00:01  381k
100 1118k  100 1118k    0     0   650k      0  0:00:01  0:00:01 --:--:--  650k


In [26]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [27]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [28]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [29]:
len(df_train), len(df_val)

(73908, 61921)

In [30]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [31]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [32]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [33]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.479581745570146

In [35]:
import os
if not os.path.exists("./models"):
    os.makedirs("./models")

In [39]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    with open('models/lin_reg.bin', 'wb') as f_out:
        pickle.dump((dv, lr), f_out)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [40]:
import xgboost as xgb

C:\Users\MARCOS\anaconda3\envs\mlops-zoomcamp\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [41]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [42]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [43]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [44]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[16:58:29] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:12.52460                                                     
[1]	validation-rmse:8.86640                                                      
[2]	validation-rmse:7.48952                                                      
[3]	validation-rmse:6.98667                                                      
[4]	validation-rmse:6.79006                                                      
[5]	validation-rmse:6.70868                                                      
[6]	validation-rmse:6.66392                                                      
[7]	validation-rmse:6.64428                                                      
[8]	validation-rmse:6.62690                                                      
[9]	validation-rmse:6.62113                                                      
[10]	validation-rmse:6.61

[98]	validation-rmse:6.58603                                                     
[99]	validation-rmse:6.58622                                                     
[100]	validation-rmse:6.58598                                                    
[101]	validation-rmse:6.58569                                                    
[102]	validation-rmse:6.58626                                                    
[103]	validation-rmse:6.58653                                                    
[104]	validation-rmse:6.58755                                                    
[105]	validation-rmse:6.58749                                                    
[106]	validation-rmse:6.58770                                                    
[107]	validation-rmse:6.58725                                                    
[108]	validation-rmse:6.58767                                                    
[109]	validation-rmse:6.58732                                                    
[110]	validation

[71]	validation-rmse:6.53860                                                     
[72]	validation-rmse:6.53604                                                     
[73]	validation-rmse:6.53375                                                     
[74]	validation-rmse:6.53159                                                     
[75]	validation-rmse:6.52954                                                     
[76]	validation-rmse:6.52771                                                     
[77]	validation-rmse:6.52631                                                     
[78]	validation-rmse:6.52443                                                     
[79]	validation-rmse:6.52292                                                     
[80]	validation-rmse:6.52167                                                     
[81]	validation-rmse:6.52047                                                     
[82]	validation-rmse:6.51892                                                     
[83]	validation-

[170]	validation-rmse:6.47430                                                    
[171]	validation-rmse:6.47411                                                    
[172]	validation-rmse:6.47368                                                    
[173]	validation-rmse:6.47348                                                    
[174]	validation-rmse:6.47295                                                    
[175]	validation-rmse:6.47251                                                    
[176]	validation-rmse:6.47236                                                    
[177]	validation-rmse:6.47206                                                    
[178]	validation-rmse:6.47181                                                    
[179]	validation-rmse:6.47150                                                    
[180]	validation-rmse:6.47133                                                    
[181]	validation-rmse:6.47107                                                    
[182]	validation

[269]	validation-rmse:6.45455                                                    
[270]	validation-rmse:6.45439                                                    
[271]	validation-rmse:6.45422                                                    
[272]	validation-rmse:6.45417                                                    
[273]	validation-rmse:6.45405                                                    
[274]	validation-rmse:6.45390                                                    
[275]	validation-rmse:6.45380                                                    
[276]	validation-rmse:6.45346                                                    
[277]	validation-rmse:6.45339                                                    
[278]	validation-rmse:6.45327                                                    
[279]	validation-rmse:6.45326                                                    
[280]	validation-rmse:6.45317                                                    
[281]	validation

...

[498]	validation-rmse:6.32551                                                    
[499]	validation-rmse:6.32547                                                    
[500]	validation-rmse:6.32517                                                    
[501]	validation-rmse:6.32500                                                    
[502]	validation-rmse:6.32493                                                    
[503]	validation-rmse:6.32472                                                    
[504]	validation-rmse:6.32454                                                    
[505]	validation-rmse:6.32446                                                    
[506]	validation-rmse:6.32433                                                    
[507]	validation-rmse:6.32408                                                    
[508]	validation-rmse:6.32411                                                    
[509]	validation-rmse:6.32424                                                    
[510]	validation

[597]	validation-rmse:6.32042                                                    
[598]	validation-rmse:6.32022                                                    
[599]	validation-rmse:6.32022                                                    
[600]	validation-rmse:6.32017                                                    
[601]	validation-rmse:6.32050                                                    
[602]	validation-rmse:6.32056                                                    
[603]	validation-rmse:6.32062                                                    
[604]	validation-rmse:6.32062                                                    
[605]	validation-rmse:6.32064                                                    
[606]	validation-rmse:6.32061                                                    
[607]	validation-rmse:6.32076                                                    
[608]	validation-rmse:6.32085                                                    
[609]	validation

[696]	validation-rmse:6.31943                                                    
[697]	validation-rmse:6.31942                                                    
[698]	validation-rmse:6.31949                                                    
[699]	validation-rmse:6.31966                                                    
[700]	validation-rmse:6.31960                                                    
[701]	validation-rmse:6.31950                                                    
[702]	validation-rmse:6.31948                                                    
[703]	validation-rmse:6.31942                                                    
[704]	validation-rmse:6.31932                                                    
[705]	validation-rmse:6.31919                                                    
[706]	validation-rmse:6.31923                                                    
[707]	validation-rmse:6.31917                                                    
[708]	validation

[72]	validation-rmse:6.60358                                                     
[73]	validation-rmse:6.60310                                                     
[74]	validation-rmse:6.60208                                                     
[75]	validation-rmse:6.60083                                                     
[76]	validation-rmse:6.60071                                                     
[77]	validation-rmse:6.59973                                                     
[78]	validation-rmse:6.59968                                                     
[79]	validation-rmse:6.59912                                                     
[80]	validation-rmse:6.59904                                                     
[81]	validation-rmse:6.59858                                                     
[82]	validation-rmse:6.59752                                                     
[83]	validation-rmse:6.59715                                                     
[84]	validation-

[171]	validation-rmse:6.58910                                                    
[172]	validation-rmse:6.58901                                                    
[173]	validation-rmse:6.58898                                                    
[174]	validation-rmse:6.58889                                                    
[175]	validation-rmse:6.58884                                                    
[176]	validation-rmse:6.58843                                                    
[177]	validation-rmse:6.58888                                                    
[178]	validation-rmse:6.58900                                                    
[179]	validation-rmse:6.58893                                                    
[180]	validation-rmse:6.58862                                                    
[181]	validation-rmse:6.58864                                                    
[182]	validation-rmse:6.58993                                                    
[183]	validation

In [45]:
mlflow.xgboost.autolog(disable=True)

In [46]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[17:35:21] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48427
[1]	validation-rmse:17.95632
[2]	validation-rmse:16.59113
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32801
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47582
[10]	validation-rmse:9.96482
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12621
[13]	validation-rmse:8.78521
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23331
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65208
[19]	validation-rmse:7.50742
[20]	validation-rmse:7.38234
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	validation-rmse:6.86273
[28]	validation-rmse:6.82110
[29]

[272]	validation-rmse:6.38727
[273]	validation-rmse:6.38673
[274]	validation-rmse:6.38660
[275]	validation-rmse:6.38649
[276]	validation-rmse:6.38615
[277]	validation-rmse:6.38595
[278]	validation-rmse:6.38584
[279]	validation-rmse:6.38558
[280]	validation-rmse:6.38529
[281]	validation-rmse:6.38509
[282]	validation-rmse:6.38479
[283]	validation-rmse:6.38455
[284]	validation-rmse:6.38428
[285]	validation-rmse:6.38410
[286]	validation-rmse:6.38377
[287]	validation-rmse:6.38341
[288]	validation-rmse:6.38321
[289]	validation-rmse:6.38288
[290]	validation-rmse:6.38241
[291]	validation-rmse:6.38218
[292]	validation-rmse:6.38186
[293]	validation-rmse:6.38155
[294]	validation-rmse:6.38099
[295]	validation-rmse:6.38072
[296]	validation-rmse:6.38058
[297]	validation-rmse:6.38034
[298]	validation-rmse:6.38019
[299]	validation-rmse:6.38001
[300]	validation-rmse:6.37985
[301]	validation-rmse:6.37969
[302]	validation-rmse:6.37924
[303]	validation-rmse:6.37890
[304]	validation-rmse:6.37895
[305]	vali

[546]	validation-rmse:6.33694
[547]	validation-rmse:6.33680
[548]	validation-rmse:6.33659
[549]	validation-rmse:6.33672
[550]	validation-rmse:6.33653
[551]	validation-rmse:6.33657
[552]	validation-rmse:6.33651
[553]	validation-rmse:6.33631
[554]	validation-rmse:6.33623
[555]	validation-rmse:6.33601
[556]	validation-rmse:6.33609
[557]	validation-rmse:6.33599
[558]	validation-rmse:6.33582
[559]	validation-rmse:6.33582
[560]	validation-rmse:6.33562
[561]	validation-rmse:6.33556
[562]	validation-rmse:6.33554
[563]	validation-rmse:6.33540
[564]	validation-rmse:6.33525
[565]	validation-rmse:6.33519
[566]	validation-rmse:6.33517
[567]	validation-rmse:6.33522
[568]	validation-rmse:6.33518
[569]	validation-rmse:6.33510
[570]	validation-rmse:6.33491
[571]	validation-rmse:6.33472
[572]	validation-rmse:6.33458
[573]	validation-rmse:6.33443
[574]	validation-rmse:6.33427
[575]	validation-rmse:6.33401
[576]	validation-rmse:6.33395
[577]	validation-rmse:6.33392
[578]	validation-rmse:6.33378
[579]	vali

[820]	validation-rmse:6.31342
[821]	validation-rmse:6.31328
[822]	validation-rmse:6.31317
[823]	validation-rmse:6.31322
[824]	validation-rmse:6.31316
[825]	validation-rmse:6.31318
[826]	validation-rmse:6.31317
[827]	validation-rmse:6.31316
[828]	validation-rmse:6.31309
[829]	validation-rmse:6.31290
[830]	validation-rmse:6.31278
[831]	validation-rmse:6.31271
[832]	validation-rmse:6.31266
[833]	validation-rmse:6.31270
[834]	validation-rmse:6.31273
[835]	validation-rmse:6.31271
[836]	validation-rmse:6.31268
[837]	validation-rmse:6.31259
[838]	validation-rmse:6.31253
[839]	validation-rmse:6.31239
[840]	validation-rmse:6.31230
[841]	validation-rmse:6.31226
[842]	validation-rmse:6.31215
[843]	validation-rmse:6.31216
[844]	validation-rmse:6.31220
[845]	validation-rmse:6.31214
[846]	validation-rmse:6.31200
[847]	validation-rmse:6.31178
[848]	validation-rmse:6.31168
[849]	validation-rmse:6.31149
[850]	validation-rmse:6.31131
[851]	validation-rmse:6.31137
[852]	validation-rmse:6.31138
[853]	vali

C:\Users\MARCOS\anaconda3\envs\mlops-zoomcamp\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [47]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

C:\Users\MARCOS\anaconda3\envs\mlops-zoomcamp\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
